# Age prediction from MRI features

In [13]:
# libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utils import visualize, create_dataset_age, create_dataset_mri, cv
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression, mutual_info_regression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso

In [14]:
import warnings
warnings.filterwarnings('ignore')

## Create dataset

In [15]:
target = 'Age'
# use all MRI high-level features, no DTI
data = create_dataset_mri(SCORE = target)
# remove the diagnosis colums
data.drop(columns=['DX_01_Cat', 'DX_01', 'DX_01_Sub'], inplace=True)

In [16]:
test_indices = pd.read_csv('data/test_IDS.csv')
# Separate test and train set
data_test = pd.merge(data, test_indices, on='ID', how='inner')
data_train = data.loc[~data['ID'].isin(list((set(test_indices['ID']))))]

In [17]:
# labels and features
y_train = data_train[target]
X_train = data_train.drop([target, 'ID'], axis=1)
y_test = data_test[target]
X_test = data_test.drop([target, 'ID'], axis=1)

In [18]:
column_names = X_train.columns

## Fix a baseline (mean age)

In [19]:
# Baseline
mean = np.mean(y_train)
print('mean_age = {}'.format(mean))
baseline_MSE = sum((mean - y_train)**2)/len(y_train)
print('baseline_MSE = {}'.format(baseline_MSE))

mean_age = 10.696383804093566
baseline_MSE = 13.820677264194652


# Explore different regressors and feature selection procedures

## SVR with no feature selection

In [23]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('regression', SVR())])
# Prepare sets of parameters for gridsearch
parameters = {'regression__C' : [0.5, 1],
            'regression__epsilon' : [0.1, 0.5, 0.7],
            'regression__kernel' : ['rbf']}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# inner CV (model selection)
model_fitted = model.fit(X_train, y_train)
# see what has been chosen     
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
# Outer CV
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2=True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    3.6s finished


Selected hyperparameters: {'regression__C': 0.5, 'regression__epsilon': 0.7, 'regression__kernel': 'rbf'}
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.0s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


...

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.1s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.1s finished



Average expected test MSE: 6.3367123202370905

Average expected test r2: 0.5372961548752249
True test error: 6.892941255706891
Test r2: 0.4636394081875016


## SelectKBest + SVR


In [24]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('feat_select', SelectKBest()),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'regression__C' : [1],
            'regression__epsilon' : [0.1, 0.5, 0.7],
            'regression__kernel' : ['rbf'],
            'feat_select__score_func' : [f_regression],
            'feat_select__k' : [10, 20, 40, 80, 95]}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# inner CV (model selection)
model_fitted = model.fit(X_train, y_train)
# see what has been chosen   
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    3.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Selected hyperparameters: {'feat_select__k': 10, 'feat_select__score_func': <function f_regression at 0x7efc2e8bd7b8>, 'regression__C': 1, 'regression__epsilon': 0.1, 'regression__kernel': 'rbf'}
Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  52 out of  75 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 15 candidates, totalling 75 fits


...

[Parallel(n_jobs=-1)]: Done  52 out of  75 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    0.7s finished


Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 out of  75 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 15 candidates, totalling 75 fits

Average expected test MSE: 7.612374081094432

Average expected test r2: 0.4429164285794074
True test error: 9.588931863482319
Test r2: 0.2538562308377518


[Parallel(n_jobs=-1)]: Done  52 out of  75 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    0.7s finished


## ExtraTrees + SVR

In [25]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor(100))),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'regression__C' : [0.3, 1, 1.5],
            'regression__epsilon' : [0.05, 0.1, 0.5],
            'regression__kernel' : ['rbf'],
            'feat_select__estimator__n_estimators' : [100],}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# inner CV (model selection)
model_fitted = model.fit(X_train, y_train)
# see what has been chosen  
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   34.0s finished


Selected hyperparameters: {'feat_select__estimator__n_estimators': 100, 'regression__C': 0.3, 'regression__epsilon': 0.05, 'regression__kernel': 'rbf'}
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   25.3s finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


...

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   25.3s finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   25.4s finished



Average expected test MSE: 5.962987882073295

Average expected test r2: 0.5643137725743459
True test error: 6.617686216536095
Test r2: 0.48505783469545827


## RandomForest

In [26]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('regression', RandomForestRegressor())
])
# Prepare sets of parameters for gridsearch
parameters = {'regression__max_depth' : [2, 3, 6, 12],
            'regression__min_samples_leaf' : [1, 3, 5, 8]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# inner CV (model selection)
model_fitted = model.fit(X_train, y_train)
# see what has been chosen  
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    7.6s finished


Selected hyperparameters: {'regression__max_depth': 2, 'regression__min_samples_leaf': 5}
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    6.0s finished


Fitting 5 folds for each of 16 candidates, totalling 80 fits


...

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    5.9s finished


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    5.9s finished



Average expected test MSE: 8.613366879987769

Average expected test r2: 0.3720121409980696
True test error: 9.207936371549037
Test r2: 0.28350264155712945


## ExtraTrees + XGBoost

In [28]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor())),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'feat_select__estimator__n_estimators' : [80, 100, 120],
              'feat_select__estimator__min_samples_leaf' : [1, 2, 4, 5, 6],
              'regression__booster__alpha' : [0.05, 0.2, 0.5]
              #'regression__booster__max_depth' : [3, 4, 5, 6, 7]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# inner CV (model selection)
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2= True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  2.1min finished


[11:16:15] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'feat_select__estimator__min_samples_leaf': 1, 'feat_select__estimator__n_estimators': 80, 'regression__booster__alpha': 0.05}
Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.4min finished


[11:17:40] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 45 candidates, totalling 225 fits


...

[11:20:30] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.4min finished


[11:21:56] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.4min finished


[11:23:21] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 5.416216408232089

Average expected test r2: 0.605418077993749
True test error: 6.457003321396926
Test r2: 0.49756105640210424


## Lasso

In [29]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('regression', Lasso())
])
# Prepare sets of parameters for gridsearch
parameters = {'regression__alpha' : [1, 1.2, 1.4, 1.7, 1.9, 2, 2.3, 2.5, 3, 3.5, 4, 6],
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# inner CV (model selection)
model_fitted = model.fit(X_train, y_train)
# see what has been chosen     
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2=True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Selected hyperparameters: {'regression__alpha': 2}
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 12 candidates, totalling 60 fits


...

[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 12 candidates, totalling 60 fits

Average expected test MSE: 13.915182350414664

Average expected test r2: -0.01933891037082023
True test error: 12.91713560677713
Test r2: -0.005121361340075126


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.5s finished


## SelectKBEST + XGBoost

In [30]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('feat_select', SelectKBest()),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'feat_select__score_func' : [f_regression],
            'feat_select__k' : [10, 30, 60, 100, 120, 160],
              'regression__booster__alpha' : [0, 0.05, 0.1, 0.2, 0.6, 1],
              'regression__booster__max_depth' : [3, 4, 5, 6, 7]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# inner CV (model selection)
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
# outer CV (model evaluation)
(estimated_test_error), r2 = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  1.0min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[11:24:27] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'feat_select__k': 10, 'feat_select__score_func': <function f_regression at 0x7efc2e8bd7b8>, 'regression__booster__alpha': 0, 'regression__booster__max_depth': 3}
Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 610 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:   51.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 605 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:   51.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[11:27:53] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 604 tasks      | elapsed:   23.4s


[11:28:44] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 8.013956709809241

Average expected test r2: 0.41448560657781985
True test error: 8.671673582626594
Test r2: 0.32523086992340544


[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:   51.2s finished


# MRI + DTI

In [31]:
target = 'Age'
# use all MRI high-level features, no DTI
data = create_dataset_mri(SCORE = target, DTI = True)
# for the moment, remove the diagnosis colums
try:
    data.drop(columns=['DX_01_Cat'], inplace=True)
except:
    pass
try:
    data.drop(columns=['DX_01_Sub'], inplace=True)
except:
    pass
try:
    data.drop(columns=['DX_01'], inplace=True)
except:
    pass

# labels and features
y = data[target]
X = data.drop([target, 'ID'], axis=1)
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## ExtraTrees + SVR

In [33]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor(100))),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'regression__C' : [0.3, 0.5, 0.8, 1, 2],
            'regression__epsilon' : [0.05, 0.1, 0.5],
            'regression__kernel' : ['rbf'],
            'feat_select__estimator__n_estimators' : [100],
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# inner CV (model selection)
model_fitted = model.fit(X_train, y_train)
# see what has been chosen  
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.8min finished


Selected hyperparameters: {'feat_select__estimator__n_estimators': 100, 'imputation__strategy': 'most_frequent', 'regression__C': 0.3, 'regression__epsilon': 0.5, 'regression__kernel': 'rbf'}
Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.3min finished


Fitting 5 folds for each of 45 candidates, totalling 225 fits


...

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   59.8s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.3min finished


Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.3min finished



Average expected test MSE: 6.8465783509529015

Average expected test r2: 0.5102938362726915
True test error: 6.730544787161029
Test r2: 0.5265380310701002


## ExtraTrees + XGBoost

In [34]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor())),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'feat_select__estimator__n_estimators' : [100],
              'feat_select__estimator__min_samples_leaf' : [1, 2, 4, 5, 6],
              'regression__booster__alpha' : [0.05, 0.2, 0.5]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# inner CV (model selection)
model_fitted = model.fit(X_train, y_train)
# see what has been chosen  
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   17.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   57.3s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.1min finished


[11:47:47] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'feat_select__estimator__min_samples_leaf': 1, 'feat_select__estimator__n_estimators': 100, 'imputation__strategy': 'most_frequent', 'regression__booster__alpha': 0.05}
Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:   50.2s finished


[11:48:39] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 45 candidates, totalling 225 fits


...

[11:50:24] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:   50.1s finished


[11:51:15] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   43.2s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:   50.6s finished


[11:52:07] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 5.433635422756599

Average expected test r2: 0.6062180570356605
True test error: 5.379373626095754
Test r2: 0.6215865269213704


## ExtraTrees + RandomForest

In [ ]:
pipe = Pipeline([('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor())),
  ('regression', RandomForestRegressor())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'feat_select__estimator__n_estimators' : [5100],
              'feat_select__estimator__min_samples_leaf' : [1, 5, 8],
            'regression__max_depth' : [2, 3, 6],
            'regression__min_samples_leaf' : [1, 3, 5, 8]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# inner CV (model selection)
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 70.0min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 107.1min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 121.0min finished


Selected hyperparameters: {'feat_select__estimator__min_samples_leaf': 5, 'feat_select__estimator__n_estimators': 5100, 'imputation__strategy': 'median', 'regression__max_depth': 2, 'regression__min_samples_leaf': 1}
Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 51.2min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 79.2min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 89.8min finished


Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 51.1min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 79.2min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 89.7min finished


Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 51.5min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 79.6min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 90.3min finished


Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 10.2min
